In [1]:
import pandas as pd

## select first RNAs

In [2]:
out = open("topRNAs.fna", "w")
names = []

with open("GCF_000001405.40_GRCh38.p14_rna.fna") as f:
    chosen_rnas = 0
    writing = True
    while chosen_rnas < 20:
        line = f.readline()
        if line[0] == ">":
            name = line.split()[0]
            if name not in names:
                chosen_rnas += 1
                writing = True
                names.append(name)
            else:
                writing = False
        if writing:
            out.write(line)
    print(names)

out.flush()
out.close()


['>NM_000014.6', '>NM_000015.3', '>NM_000016.6', '>NM_000017.4', '>NM_000018.4', '>NM_000019.4', '>NM_000020.3', '>NM_000021.4', '>NM_000022.4', '>NM_000023.4', '>NM_000024.6', '>NM_000025.3', '>NM_000026.4', '>NM_000027.4', '>NM_000028.2', '>NM_000030.3', '>NM_000031.6', '>NM_000032.5', '>NM_000033.4', '>NM_000035.4']


In [10]:
%%bash
blastn_cmd.sh

## parsing BLASTn output

In [3]:
blastn_out = open("blastn.out")
blast_res = pd.read_csv("blastn.out", sep="\t", header=None)
blast_res.columns = ["NM", "NC", "f3", "f4", "f5", "f6", "q_start", "q_end", "start", "end", "e_value", "f12"]
blast_res

,NM,NC,f3,f4,f5,f6,q_start,q_end,start,end,e_value,f12
0,NM_000014.6,NC_000012.12,99.573,234,1,0,2311,2544,9091429,9091196,2.690000e-116,427.0
1,NM_000014.6,NC_000012.12,99.571,233,0,1,1332,1564,9101678,9101447,3.480000e-115,424.0
2,NM_000014.6,NC_000012.12,100.000,228,0,0,3600,3827,9074786,9074559,1.250000e-114,422.0
3,NM_000014.6,NC_000012.12,100.000,222,0,0,3824,4045,9072874,9072653,2.710000e-111,411.0
4,NM_000014.6,NC_000012.12,98.291,234,4,0,2311,2544,9165367,9165134,2.710000e-111,411.0
...,...,...,...,...,...,...,...,...,...,...,...,...
22109,NM_000033.4,NC_000023.11,100.000,148,0,0,1490,1637,153736109,153736256,2.940000e-70,274.0
22110,NM_000033.4,NC_000023.11,100.000,148,0,0,1899,2046,153740091,153740238,2.940000e-70,274.0
22111,NM_000033.4,NC_000023.11,99.248,133,1,0,2271,2403,153743215,153743347,2.990000e-60,241.0
22112,NM_000033.4,NC_000023.11,100.000,99,0,0,1803,1901,153737155,153737253,5.110000e-43,183.0


## extract RNAs annotations

In [4]:
def parse_gff_entry(entry):
    fields = entry.split("\t")
    descr = fields[-1]
    if "NM_0000" not in descr:
        return
    nm = "NM_0000" + descr.split("NM_0000")[1][:4]
    if nm in nm_names:
        annot.loc[len(annot)] = fields
        print(annot.shape, end="\r")

In [5]:
gff = open("GCF_000001405.40_GRCh38.p14_genomic.gff")
annot = pd.DataFrame(columns=["NC", "source", "feature", "start", "end", "score", "strand", "frame", "descr"])
nc_names = set(blast_res["NC"])
nm_names = set(blast_res["NM"])
for line in gff:
    if line.startswith("#"):
        continue
    parse_gff_entry(line[:-1])
gff.close()
annot.to_csv("annotations.csv")
annot

,NC,source,feature,start,end,score,strand,frame,descr
0,NC_000001.11,BestRefSeq,mRNA,75724709,75763679,.,+,.,ID=rna-NM_000016.6;Parent=gene-ACADM;Dbxref=En...
1,NC_000001.11,BestRefSeq,exon,75724709,75724817,.,+,.,ID=exon-NM_000016.6-1;Parent=rna-NM_000016.6;D...
2,NC_000001.11,BestRefSeq,exon,75728401,75728488,.,+,.,ID=exon-NM_000016.6-2;Parent=rna-NM_000016.6;D...
3,NC_000001.11,BestRefSeq,exon,75732644,75732741,.,+,.,ID=exon-NM_000016.6-3;Parent=rna-NM_000016.6;D...
4,NC_000001.11,BestRefSeq,exon,75732853,75732922,.,+,.,ID=exon-NM_000016.6-4;Parent=rna-NM_000016.6;D...
...,...,...,...,...,...,...,...,...,...
484,NC_000023.11,BestRefSeq,CDS,153740092,153740237,.,+,0,ID=cds-NP_000024.2;Parent=rna-NM_000033.4;Dbxr...
485,NC_000023.11,BestRefSeq,CDS,153740574,153740719,.,+,1,ID=cds-NP_000024.2;Parent=rna-NM_000033.4;Dbxr...
486,NC_000023.11,BestRefSeq,CDS,153742987,153743071,.,+,2,ID=cds-NP_000024.2;Parent=rna-NM_000033.4;Dbxr...
487,NC_000023.11,BestRefSeq,CDS,153743221,153743346,.,+,1,ID=cds-NP_000024.2;Parent=rna-NM_000033.4;Dbxr...


In [6]:
annot = pd.read_csv("annotations.csv")

In [7]:
annot["NM"] = annot["descr"].apply(lambda x: x.split("Genbank:")[1][:11])
annot

,Unnamed: 0,NC,source,feature,start,end,score,strand,frame,descr,NM
0,0,NC_000001.11,BestRefSeq,mRNA,75724709,75763679,.,+,.,ID=rna-NM_000016.6;Parent=gene-ACADM;Dbxref=En...,NM_000016.6
1,1,NC_000001.11,BestRefSeq,exon,75724709,75724817,.,+,.,ID=exon-NM_000016.6-1;Parent=rna-NM_000016.6;D...,NM_000016.6
2,2,NC_000001.11,BestRefSeq,exon,75728401,75728488,.,+,.,ID=exon-NM_000016.6-2;Parent=rna-NM_000016.6;D...,NM_000016.6
3,3,NC_000001.11,BestRefSeq,exon,75732644,75732741,.,+,.,ID=exon-NM_000016.6-3;Parent=rna-NM_000016.6;D...,NM_000016.6
4,4,NC_000001.11,BestRefSeq,exon,75732853,75732922,.,+,.,ID=exon-NM_000016.6-4;Parent=rna-NM_000016.6;D...,NM_000016.6
...,...,...,...,...,...,...,...,...,...,...,...
484,484,NC_000023.11,BestRefSeq,CDS,153740092,153740237,.,+,0,ID=cds-NP_000024.2;Parent=rna-NM_000033.4;Dbxr...,NP_000024.2
485,485,NC_000023.11,BestRefSeq,CDS,153740574,153740719,.,+,1,ID=cds-NP_000024.2;Parent=rna-NM_000033.4;Dbxr...,NP_000024.2
486,486,NC_000023.11,BestRefSeq,CDS,153742987,153743071,.,+,2,ID=cds-NP_000024.2;Parent=rna-NM_000033.4;Dbxr...,NP_000024.2
487,487,NC_000023.11,BestRefSeq,CDS,153743221,153743346,.,+,1,ID=cds-NP_000024.2;Parent=rna-NM_000033.4;Dbxr...,NP_000024.2


## filtering BLASTn alignments

In [8]:
joined = pd.merge(blast_res, annot, on=["NC", "NM"])
joined.drop(["descr", "f3", "f4", "f5", "f6", "f12", "Unnamed: 0" ]
            , axis=1, inplace=True)
joined["start_y"] = pd.to_numeric(joined["start_y"])
joined["end_y"] = pd.to_numeric(joined["end_y"])

joined = joined[ (joined["start_x"] < joined["start_y"]) & (joined["end_x"] > joined["end_y"]) ]
joined.head()

,NM,NC,q_start,q_end,start_x,end_x,e_value,source,feature,start_y,end_y,score,strand,frame
1487,NM_000016.6,NC_000001.11,786,931,75749417,75749562,2.340000e-69,BestRefSeq,exon,75749419,75749559,.,+,.
1525,NM_000016.6,NC_000001.11,678,788,75745805,75745915,6.680000e-50,BestRefSeq,exon,75745806,75745914,.,+,.
1588,NM_000016.6,NC_000001.11,457,561,75734780,75734886,1.900000e-35,BestRefSeq,exon,75734791,75734871,.,+,.
1632,NM_000017.4,NC_000012.12,92,274,120727012,120727193,8.720000e-83,BestRefSeq,exon,120727026,120727189,.,+,.
1646,NM_000017.4,NC_000012.12,530,685,120737834,120737989,5.290000e-75,BestRefSeq,exon,120737837,120737988,.,+,.


In [10]:
joined = joined.sort_values("q_start").sort_values("NM").head()
joined.rename({"start_x": "align_start",
                    "end_x": "align_end",
                    "start_y": "annot_start",
                    "end_y": "annot_end"}, axis=1, inplace=True)
joined

,NM,NC,q_start,q_end,align_start,align_end,e_value,source,feature,annot_start,annot_end,score,strand,frame
1588,NM_000016.6,NC_000001.11,457,561,75734780,75734886,1.900000e-35,BestRefSeq,exon,75734791,75734871,.,+,.
1525,NM_000016.6,NC_000001.11,678,788,75745805,75745915,6.680000e-50,BestRefSeq,exon,75745806,75745914,.,+,.
1487,NM_000016.6,NC_000001.11,786,931,75749417,75749562,2.340000e-69,BestRefSeq,exon,75749419,75749559,.,+,.
1670,NM_000017.4,NC_000012.12,855,1006,120738532,120738683,4.140000e-66,BestRefSeq,exon,120738533,120738670,.,+,.
1704,NM_000017.4,NC_000012.12,991,1090,120738817,120738916,7.130000e-44,BestRefSeq,exon,120738820,120738915,.,+,.


In [11]:
joined.to_csv("results.csv")